In [1]:
import os 
import numpy as np
import pandas as pd
import scipy.io
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
from neo.core import SpikeTrain
from quantities import ms, s, Hz
from elephant.statistics import mean_firing_rate
from elephant.statistics import time_histogram, instantaneous_rate
from elephant.kernels import GaussianKernel

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
file_to_use = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Tiffany\\data.csv'

In [4]:
df = pd.read_csv(file_to_use, sep=',', index_col=0)

In [5]:
df

,cluster_id,fixed_times,trial,Delay_ON,Delay_OFF,Delay_length,Stimulus_ON,Response_ON,START,END,a_Stimulus_ON,a_Delay_ON,a_Delay_OFF,a_Response_ON,vector_answer,reward_side,hit
0,1193,88.264567,1,89.061367,89.161400,0.100033,88.661367,89.361400,88.161367,95.522933,-0.396800,-0.796800,-0.896833,-1.096833,0,0.0,1.0
1,1193,88.419400,1,89.061367,89.161400,0.100033,88.661367,89.361400,88.161367,95.522933,-0.241967,-0.641967,-0.742000,-0.942000,0,0.0,1.0
2,1193,88.453467,1,89.061367,89.161400,0.100033,88.661367,89.361400,88.161367,95.522933,-0.207900,-0.607900,-0.707933,-0.907933,0,0.0,1.0
3,1193,88.669733,1,89.061367,89.161400,0.100033,88.661367,89.361400,88.161367,95.522933,0.008367,-0.391633,-0.491667,-0.691667,0,0.0,1.0
4,1193,88.820100,1,89.061367,89.161400,0.100033,88.661367,89.361400,88.161367,95.522933,0.158733,-0.241267,-0.341300,-0.541300,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265735,943,2626.225300,222,2622.972567,2625.972633,3.000067,2622.572567,2626.172633,2622.072567,NaN,3.652733,3.252733,0.252667,0.052667,0,1.0,0.0
265736,943,2626.795500,222,2622.972567,2625.972633,3.000067,2622.572567,2626.172633,2622.072567,NaN,4.222933,3.822933,0.822867,0.622867,0,1.0,0.0
265737,943,2627.401300,222,2622.972567,2625.972633,3.000067,2622.572567,2626.172633,2622.072567,NaN,4.828733,4.428733,1.428667,1.228667,0,1.0,0.0
265738,943,2628.004167,222,2622.972567,2625.972633,3.000067,2622.572567,2626.172633,2622.072567,NaN,5.431600,5.031600,2.031533,1.831533,0,1.0,0.0


In [6]:
df.cluster_id.unique()

array([1193, 1794, 1215,  301, 1402, 1621,  294, 1542, 1548, 1759, 1574,
        474, 1708, 1713,  785, 1791, 1399, 1406, 1441, 1097, 1430, 1386,
       1611, 1761, 1375, 1749, 1145, 1688, 1709, 1479, 1035, 1073, 1380,
       1763, 1352, 1231, 1698, 1788, 1225,   21,  231, 1766,  578,  606,
       1203,  943,  648,  895,  590,  520, 1426, 1540, 1774,  909],
      dtype=int64)

In [15]:
nx = df.loc[df['cluster_id']==1794]
nxt = nx.loc[df['trial']==1]

In [19]:
nxt.START.unique()[0]

88.16136666666668

In [17]:
times_spikes = nxt.fixed_times.values
times_spikes

array([88.8387    , 89.5898    , 89.89163333, 89.91333333, 90.95413333,
       91.0568    , 91.23436667, 91.2446    , 91.26063333, 91.27296667,
       91.2757    , 91.305     , 91.3226    , 91.33256667, 91.40486667,
       91.41536667, 91.432     , 91.45806667, 91.47376667, 91.48363333,
       91.4974    , 91.52263333, 91.6043    , 91.62506667, 91.72543333,
       91.76506667, 91.8417    , 92.82813333, 92.85433333, 92.88513333,
       92.90356667, 92.98276667, 93.00916667, 93.76556667, 93.81343333,
       93.83466667, 93.87553333, 93.91053333, 93.94786667, 94.09096667,
       94.12296667, 94.15823333, 94.16796667, 94.19033333, 94.30716667,
       94.3202    , 94.3749    , 94.44686667, 94.4721    , 94.51106667])

In [22]:
stop_time =  times_spikes.max() ##Needed to make the spiketrain
start_time = nxt.START.unique()[0]*ms #has de ponerlo si no se empieza a samplear alineado al mismo momento, default es 0